In [1]:
!pip install langchain
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install sentence-transformers
!pip install faiss-gpu


from langchain import hub
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/1

In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "nachtwindecho/mistralai-Code-Instruct-Finetune-SG1-V4"

tokenizer = AutoTokenizer.from_pretrained(model_id)
llm = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map='auto')


tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [78]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from google.colab import drive
drive.mount("/content/drive")

loader = TextLoader("/content/drive/MyDrive/Target/data.txt", encoding = 'UTF-8')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
chunk_size=200, chunk_overlap=50, separators=[" ", ",", "\n"]
)

#text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=25)
texts = text_splitter.split_documents(documents)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
# add a retriever for RAG here
modelPath = "intfloat/e5-large-unsupervised"
embeddings = HuggingFaceEmbeddings(model_name = modelPath,  model_kwargs = {'device':'cuda'},encode_kwargs={'normalize_embeddings':False})

# Using faiss index
from langchain.vectorstores import FAISS
db = FAISS.from_documents(texts, embeddings)

retriever = db.as_retriever()

In [41]:
import re

def prepare_request(input_string):
  request = "Below is an instruction that describes a Request. Write a Response that completes the Request. \n\n Request: " + input_string
  return request

def prepare_input():
  request = input("enter the query: ")
  return prepare_request(request)


def contains_inst(sentence):
    return "[/INST]" in sentence#


def extract_answer(input_string):
    response = re.split('Helpful Answer:', input_string)[-1].strip()
    separator = "[/INST]"

    if contains_inst(sentence=separator):
      split_text = response.split(separator)
      formatted_text = ''.join(split_text)
      return formatted_text
    else:
      return response


def extract_single_line_answer(input_string):
    response = re.split('Helpful Answer:', input_string)[-1].strip()
    separator = "[/INST]"

    if contains_inst(sentence=separator):
      split_text = response.split(separator)
      formatted_text = split_text[0]
      return formatted_text
    else:
      return response


Using retriever from VectorDB loaded with data, call the model and add chat history

In [115]:
from langchain.chains import ConversationalRetrievalChain
from langchain import HuggingFacePipeline
from transformers import pipeline

pipeline = pipeline(
    "text-generation",
    model=llm,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

runllm = HuggingFacePipeline(
    pipeline = pipeline,
    model_kwargs={"temperature": 0.3}
)


RETRIEVAL QUERY:

In [119]:
chat_history = []
query = prepare_input()

qa = RetrievalQA.from_chain_type(llm=runllm, retriever=retriever, return_source_documents=False)
result = qa.run({"query": query})

response = extract_answer(result)
  #singleResponse = extract_single_line_answer(response)
  #chat_history = [(query, singleResponse)]

print("RESULT>" + result)
  # print("RESPONSE>" + response)

enter the query: summarize the US constitution in 5 sentences or less?
RESULT>Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

of a long-dormant genetic virus to turn the population against the ruling oligarchy.

against the Knights of the New Revelation, an extremist religious order responsible for the death of his lost love and her daughter. Because she had violated tenets about resleeving, her executioners

associates of his, the Little Blue Bugs.The Little Blue Bugs mount a semi-successful attack on a Harlan fortress and rescue Sylvie/Quellcrist. Hiding from Harlan forces in a floating base, the

tenets about resleeving, her executioners dropped her and her daughter's cortical stacks in the sea, effectively preventing them from being resleeved (into new bodies).While trying to secure passage

Question: Below is an instruction that describes a Request. Write a Response 

REGULAR QUERY:

In [121]:
# Define the langchain pipeline for llm only
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE ="""Below is an instruction that describes a Request. Write a Response that completes the Request. \n\n Request: {question} \n\n
Response:
"""

PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)
llm_chain = PROMPT | runllm
query = input("enter the query: ")
result = llm_chain.invoke({"question": query})

print(result.rstrip())

enter the query: write me a short poem about the stargate
Below is an instruction that describes a Request. Write a Response that completes the Request. 

 Request: write me a short poem about the stargate 


Response:


Stars and gate and cosmic fate 

Infinite travel and time elate 

A journey through the universe 

Our own personal starship 


A door to unknown worlds and more 

The stargate is the gate to galore 

 
A cosmic wonder 

A journey that will always endure 
 

A bridge across the cosmos 

The stargate always leaves you wanting more 

The ultimate portal 

A gate to the universe and beyond 


A gate that will always open 

A gate that never fails to please 

A gate that takes us anywhere we want to go 

The stargate is the ultimate travel guide
